In [4]:
#### IMPORT LIBRARIES
import h5py
import numpy as np

# import all skimage related stuff
import skimage.io
from skimage.measure import label
from skimage import filters, exposure, restoration
from skimage import morphology

# import all napari related stuff
import napari
from napari.types import ImageData, LabelsData, LayerDataTuple, ShapesData
from napari.layers import Image, Layer, Labels, Shapes
from magicgui import magicgui

# # import UI for stack selection
import tkinter as tk
from tkinter import filedialog

%gui qt5
import os

#### PROCESSING FUNCTIONS

def adaptive_local_threshold(image, block_size):
    # adaptive_local_threshold is a function that takes in an image and applies an odd-integer block size
    # kernel (or filter) and thresholds based on local spatial information. 
    
    return filters.threshold_local(image, block_size)

def gamma_level(image, gamma):
    # gamma_level is a function that takes in an image and changes the contrast by scaling the image 
    # by a factor "gamma".
    return exposure.adjust_gamma(image, gamma)

def global_threshold_method(image, Threshold_Method):
    # global_threshold_method is a function that allows a user to choose what kind of method to binarize
    # an image to create a mask. For a given method, a threshold will be calculated and returns a binarized 
    # image. 
    if Threshold_Method == 'None':
        pass
    
    if Threshold_Method == 'Isodata':
        thresh = filters.threshold_isodata(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Li':
        thresh = filters.threshold_li(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Mean':
        thresh = filters.threshold_mean(image) # calculate threshold using isodata method
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Minimum':
        thresh = filters.threshold_minimum(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Otsu':
        thresh = filters.threshold_otsu(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Yen':
        thresh = filters.threshold_yen(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    if Threshold_Method == 'Triangle':
        thresh = filters.threshold_triangle(image)
        tmp_img = image > thresh
        return binary_labels(tmp_img)
    
def binary_labels(image):
    # function binary_labels labels the entire neuron and entries of the Image = 2. Later, 2 => 'Dendrites'
    temp_img = image.copy()
    temp_img[temp_img > 0] = 2
    temp_img = temp_img.astype('int')
    return temp_img

def despeckle_filter(image, filter_method, radius):
    if filter_method == 'Erosion':
        tmp_img = image.copy()
        footprint = morphology.disk(radius)
        footprint = footprint[None,:,:]
        eroded = morphology.erosion(tmp_img, footprint)
        return eroded
    
    if filter_method == 'Dilation':
        tmp_img = image.copy()
        footprint = morphology.disk(radius)
        footprint = footprint[None,:,:]
        dilated = morphology.dilation(tmp_img, footprint)
        return dilated
    
    if filter_method == 'Opening':
        tmp_img = image.copy()
        footprint = morphology.disk(radius)
        footprint = footprint[None,:,:]
        opened = morphology.opening(tmp_img, footprint)
        return opened
    
    if filter_method == 'Closing':
        tmp_img = image.copy()
        footprint = morphology.disk(radius)
        footprint = footprint[None,:,:]
        closed = morphology.closing(tmp_img, footprint)
        return closed
    
#### MAIN WIDGET/PROGRAM HERE

@magicgui(
    gamma = {"widget_type": "FloatSlider", 'max': 2},
    block_size = {"widget_type": "SpinBox", 'label': 'Block Size:', 'min': 1, 'max': 20},
    threshold_method = {"choices": ['None','Isodata', 'Li', 'Mean', 'Minimum', 'Otsu', 'Triangle', 'Yen']},
    speckle_method = {"choices": ['None','Erosion', 'Dilation', 'Opening', 'Closing']},
    radius = {"widget_type": "SpinBox", 'max': 10, 'label': 'Radius'},
    layout = 'vertical'
 )

def threshold_widget(image: ImageData, 
                     gamma = 1,
                     block_size = 3,
                     threshold_method = 'None',
                     speckle_method = 'None',
                     radius = 1
                     ) -> LayerDataTuple:
    #function threshold_widget does a series of image processing and thresholding to binarize the image and make a label
    
    if image is not None:
        # adjust the gamma levelz
        label_img = gamma_level(image, gamma)
        
        # go through the stack and perform the local threshold
        for curr_stack in range(np.shape(label_img)[0]):
            label_img[curr_stack] = adaptive_local_threshold(label_img[curr_stack], block_size)
        
        # finally do a global threshold to calculate optimized value to make it black/white
        label_img = global_threshold_method(label_img, threshold_method)
        label_img = binary_labels(label_img)
        label_img = despeckle_filter(label_img, speckle_method, radius)

        return (label_img, {'name': 'neuron_label'}, 'labels')

#### WIDGET FOR PROCESSING IMAGE AND SHOWING THE PROCESSED IMAGE BEFORE SEGMENTATION
# from magicgui import widgets

@magicgui(
    filter_method = {"choices": ['None','median', 'gaussian', 'bilateral', 'TV']},
    value_slider = {"widget_type": "FloatSlider", 'max': 4, 'label': 'None'},
    layout = 'vertical'
 )
def smoothen_filter(image: ImageData,
                  filter_method = 'None',
                  value_slider = 1) -> LayerDataTuple:
    # filter_widget is a function that takes an input image and selects a filter method 
    # for denoising an image. 
    # Returns an IMAGE layer.
    
    if image is not None:
        stack_size = np.shape(image)[0]
        if filter_method == 'median': # use a median filter and go through the entire stack to apply the filter
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = filters.median(tmp_img[curr_stack], morphology.disk(value_slider))
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'gaussian': # use a gaussian filter 
            tmp_img = image.copy()
            tmp_img = filters.gaussian(tmp_img, sigma = value_slider)
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'bilateral': # use a bilateral filter 
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = restoration.denoise_bilateral(tmp_img[curr_stack], sigma_spatial = value_slider)
            
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
        
        if filter_method == 'TV': # using a total-variation (TV) denoising filter
            tmp_img = image.copy()
            for curr_stack in range(stack_size):
                tmp_img[curr_stack] = restoration.denoise_tv_chambolle(tmp_img[curr_stack], weight = value_slider)
            return (tmp_img, {'name': 'smoothened_image'}, 'image')
            
@smoothen_filter.filter_method.changed.connect
def change_label(event):
    # change_label function is written to change the label of the FloatSlider widget
    # such that the user won't be confused as to what metric is being used. 
    
    if event.value == 'median':
        filter_widget.value_slider.label = 'Pixel Radius'
    if event.value == 'gaussian':
        filter_widget.value_slider.label = 'sigma'
    if event.value == 'bilateral':
        filter_widget.value_slider.label = 'sigma_spatial'
    if event.value == 'TV':
        filter_widget.value_slider.label = 'weight'
        filter_widget.value_slider.max = 1
        filter_widget.value_slider.value = 0
        
###################################################################################

### Widget for Despeckling 
        
# @magicgui(
#     filter_method = {"choices": ['None','Erosion', 'Dilation', 'Opening', 'Closing']},
#     radius = {"widget_type": "SpinBox", 'max': 10, 'label': 'None'},
#     layout = 'vertical'
#  )

    
#####################################################################################\

### Widget for using shapes to get segmentation 
from skimage.draw import polygon

@magicgui(
    call_button = 'Generate Neuron Volume',
    layout = 'vertical'
)
def generate_neuron_volume(volume: ImageData)-> LayerDataTuple:
    
    shape_mask = viewer.layers[2].data[0]
    
#     print(shape_mask)
    tmp_projection = np.max(volume, axis=0)
    #print(volume.shape)
    #print(tmp_projection.shape)
    
    px_coord = np.zeros(tmp_projection.shape, dtype = np.uint8) # initialize map of rows and columns
    
    rr, cc = polygon(shape_mask[:,1], shape_mask[:,2]) # get the rows and columns from polygon shape
    px_coord[rr, cc] = 1 # set all the rows and columns in the matrix as 1 
    
    masked_volume = volume.copy() # copy the volume
    masked_volume = masked_volume*px_coord # element wise multiplication to filter out everything else
    return (masked_volume, {'name': 'masked_volume'}, 'image')
#     viewer.add_image(masked_volume, name = "Neuron Volume", colormap = 'red')

    
#####################################################################################

#### WIDGET FOR SAVING LAYER AS H5 FILE

@magicgui(
    call_button = 'Save Layer',
    file_picker = {"widget_type": 'FileEdit', 'value': 'N/A', 'mode': 'd'})
def save_layer(image: ImageData, label: Labels, file_picker = 'N/A'):
    folder_name = file_picker
    file_str = os.path.splitext(os.path.basename(file_path))[0]
    h5_name = file_str + '.h5'
    full_dir = os.path.join(folder_name, h5_name)
    
    # Dictionary for label ints 
    label_dict = {
            'Background' : 0,
            'Soma' : 1,
            'Dendrite' : 2,
            'Filopodia' : 3,
            'Axon' : 4,
            'Growth Cone' : 5,
            'Autofluorescence' : 6,
            'Melanocyte' : 7,
            'Noise' : 8,
        
    }
    # initialize HDF5 file
    hf =  h5py.File(full_dir, 'w')
    grp = hf.create_group("project_data")
    grp.create_dataset('raw_image', data = image.data)
    grp.create_dataset('label', data = label.data)
    grp.create_dataset('label_dict', data=label_dict)

    hf.close()
    

#####################################################################################

### Widget for Loading WIP Neurons

@magicgui(
    call_button = 'Open Files',
    file_picker = {"widget_type": 'FileEdit', 'mode': 'r'})
def open_WIP_neuron(file_picker = 'N/A'):
    neuron_file = file_picker
#     neuron_pair = h5py.File(neuron_file, 'r')


#####################################################################################

# file_path = os.path.join(neuron_dir,neuron_file)

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()
viewer = napari.Viewer()


if os.path.splitext(file_path)[1] == '.h5':
    print('pass')
    f = h5py.File(file_path, 'r')
    # load in the image and label and add to viewer
    neuron_image = np.array(edit_labels['project_data'].get('raw_image'))
    label_layer = np.array(edit_labels['project_data'].get('label'))
    viewer.add_image(neuron_image, name = 'Neuron')
    viewer.add_labels(label_layer, name = 'Neuron_label')
    f.close()
else:
    neuron_image = skimage.io.imread(file_path)
    # create a z projection of neuron volume max pixel intensities
    z_projection = neuron_image.copy()
    z_projection = np.max(neuron_image, axis=0)
    viewer.add_image(z_projection, name = 'Neuron Projection')
    viewer.add_image(neuron_image, name = 'Neuron')

viewer.window.add_dock_widget(generate_neuron_volume, name = 'Draw Volume Segment')
# viewer.window.add_dock_widget(despeckle_filter, name = 'Despeckle Filter')
viewer.window.add_dock_widget(smoothen_filter, name = 'Smoothen Filter')
viewer.window.add_dock_widget(threshold_widget, name = 'Thresholding')
viewer.window.add_dock_widget(save_layer, name = 'Save Files')
#viewer.window.add_dock_widget(open_WIP_neuron, name = 'Open WIP Neurons', area = 'left')
napari.run()

# threshold_widget()

[[  0.          58.94988299  97.6753719 ]
 [  0.          62.603231    60.41122219]
 [  0.          82.33131026  38.49113413]
 [  0.         109.36608554  56.02720458]
 [  0.         118.86479037  80.13930145]
 [  0.         124.71014718 100.59805031]
 [  0.         141.51554803 137.13153042]
 [  0.         161.97429689 164.1663057 ]
 [  0.         186.08639376 174.39568013]
 [  0.         210.19849063 175.12634973]
 [  0.         233.5799179  175.85701933]
 [  0.         256.23067557 181.70237615]
 [  0.         272.30540682 186.08639376]
 [  0.         265.7293804  166.35831451]
 [  0.         256.23067557 153.93693127]
 [  0.         253.30799716 142.24621763]
 [  0.         281.80411164 143.70755684]
 [  0.         292.03348607 153.93693127]
 [  0.         307.37754772 172.20367132]
 [  0.         301.5321909  198.507777  ]
 [  0.         300.8015213  205.81447302]
 [  0.         311.76156533 201.43045541]
 [  0.         327.10562698 198.507777  ]
 [  0.         343.18035822 188.27

In [ ]:
edit_labels = h5py.File(file_path, 'r')

In [ ]:
edit_labels.keys()

In [ ]:
edit_labels['project_data'].keys()

In [ ]:
np.array(edit_labels['project_data'].get('label'))

In [ ]:
viewer.layers[2].data